### Matrix exponential of tensor product

Let $A, B$ be two diagonalizable matrices. This means that 

\begin{align}
A = P_A D_A P_A^{-1}, \quad B = P_B D_B P_B^{-1}
\end{align}

for invertible $P_A, P_B$ and diagonal $D_A, D_B$.

Then

\begin{align}
e^{A\otimes B}
&= \sum_{k=0}^{\infty} \frac{(A\otimes B)^k}{k!} \\ 
&= (P_A \otimes P_B)\bigg[\sum_{k=0}^{\infty} \frac{(D_A\otimes D_B)^k}{k!} \bigg] (P_A\otimes P_B)^{-1} \\
&= (P_A \otimes P_B)\bigg[\sum_{k=0}^{\infty} \frac{D_A^k\otimes D_B^k}{k!} \bigg] (P_A\otimes P_B)^{-1}
\end{align}

Now, we focus on the sum of diagonals

\begin{align}
\sum_{k=0}^{\infty} \frac{D_A^k\otimes D_B^k}{k!}
&= \sum_{k=0}^{\infty} \frac{1}{k!} \begin{pmatrix}
D_{A_1}^k\cdot D_B^K & & \\
& \ddots & \\
& & D_{A_n}^k\cdot D_B^K
\end{pmatrix} \\
&= \begin{pmatrix}
\sum_{k=0}^\infty \frac{1}{k!}(D_{A_1}^k\cdot D_B^k) & & \\
& \ddots & \\
& & \sum_{k=0}^\infty \frac{1}{k!}(D_{A_n}^k\cdot D_B^k)
\end{pmatrix} \\
&= \begin{pmatrix}
e^{D_{A_1}\cdot D_B} & & \\
& \ddots & \\
& & e^{D_{A_n} \cdot D_B} 
\end{pmatrix}
\end{align}

So putting everything together, we have

\begin{align}e^{A\otimes B} = (P_A \otimes P_B)\begin{pmatrix}
e^{D_{A_1}\cdot D_B} & & \\
& \ddots & \\
& & e^{D_{A_n} \cdot D_B} 
\end{pmatrix} (P_A\otimes P_B)^{-1}
\end{align}

In [264]:
from sympy import *
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum import TensorProduct
import numpy as np
from scipy.linalg import expm

In [8]:
symbols??

In [9]:
ax, ay, az = symbols('a_x, a_y, a_z', real = True)
bx, by, bz = symbols('b_x, b_y, b_z', real = True)

X = Matrix([[0, 1], [1, 0]])
Y = Matrix([[0, -I], [I, 0]])
Z = Matrix([[1, 0], [0, -1]])

In [70]:
A = I*(ax*X + ay*Y + az*Z)

B = I*(bx*X + by*Y + bz*Z)

In [90]:
# Verify Complex Traceless Skew-Hermitian
display(A)
print(f"Skew-Hermitian: {Dagger(A) == -A}")

Matrix([
[          I*a_z, I*(a_x - I*a_y)],
[I*(a_x + I*a_y),          -I*a_z]])

Skew-Hermitian: True


In [135]:
# Diagonalize
PA, DA = A.diagonalize()

print("PA")
display(PA)
print("DA")
display(simplify(DA))

PA


Matrix([
[(a_z - sqrt(a_x**2 + a_y**2 + a_z**2))/(a_x + I*a_y), (a_z + sqrt(a_x**2 + a_y**2 + a_z**2))/(a_x + I*a_y)],
[                                                   1,                                                    1]])

DA


Matrix([
[-I*sqrt(a_x**2 + a_y**2 + a_z**2),                               0],
[                                0, sqrt(-a_x**2 - a_y**2 - a_z**2)]])

In [144]:
#Verify correctness of diagonalization
simplify(PA@DA@PA.inv()) == A.expand()

True

In [157]:
# Rearrange eigenvectors and eigenvalues so the matrix looks nicer
a = sqrt(ax**2 + ay**2 + az**2)
PA = Matrix([[az + a, az-a], 
             [ax+I*ay, ax+I*ay]])
DA = I*a*Z

In [158]:
#Verify correctness of diagonalization
simplify(PA@DA@PA.inv()) == A.expand()

True

In [161]:
# Rearrange eigenvectors and eigenvalues so the matrix looks nicer
b = sqrt(bx**2 + by**2 + bz**2)
PB = Matrix([[bz+b, bz-b], 
             [bx+I*by, bx+I*by]])
DB = I*b*Z

In [162]:
#Verify correctness of diagonalization
simplify(PB@DB@PB.inv()) == B.expand()

True

### Computing the Tensor Product

In [183]:
# Declare A, B in SU(2)
A = I*(ax*X + ay*Y + az*Z)
B = I*(bx*X + by*Y + bz*Z)

# Declare shorthand
a = sqrt(ax**2 + ay**2 + az**2)
b = sqrt(bx**2 + by**2 + bz**2)

# Diagonal Matrices
DA, DB = I*a*Z, I*b*Z

# Invertible Matrices
PA = Matrix([[az + a, az-a], 
             [ax+I*ay, ax+I*ay]])
PB = Matrix([[bz+b, bz-b], 
             [bx+I*by, bx+I*by]])

In [190]:
#Verify correctness of diagonalization
print('A', end = ': ')
print(simplify(PA@DA@PA.inv()) == A.expand())

#Verify correctness of diagonalization
print('B', end = ': ')
print(simplify(PB@DB@PB.inv()) == B.expand())

A: True
B: True


We have that $SU(2) \otimes SU(2) = \exp(\mathfrak{su}(2)) \otimes \exp(\mathfrak{su}(2))$. Hence, we are going to take the matrix exponential first then the tensor product.